### Importing Libraries 

In [28]:
import tensorflow as tf
import keras
import sklearn
import numpy as np
import pandas as pd
import glob
import os

/home/ram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ram/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:472: 

### Loading Input .csv into 2D array

In [10]:
dataset_X2 = np.loadtxt('data_files/input.csv', delimiter = ',', skiprows = 1)

### Loading Output.csv into 3D array

In [11]:
folder_name = 'data_files'
file_type = 'csv'
seperator = ','
dataset_Y27 = np.dstack([pd.read_csv(f, sep = seperator) for f in glob.glob(folder_name + "/output_*." + file_type)]) #(257, 7, 14641)
dataset_Y26 = np.delete(dataset_Y27, 6, axis = 1) #(257, 6, 14641)

### Writing the dataset in h5py file

In [14]:
import h5py
h5f = h5py.File('data.h5', 'w')
h5f.create_dataset('sizes', data = dataset_X2)
dataset_Y2 = dataset_Y26.tolist()
h5f.create_dataset('spectrum', data = dataset_Y2)
h5f.close()

### Reading the h5py file for X and Y values

In [26]:
h5f = h5py.File('./data.h5','r')
X = h5f['sizes'][:]
YYY = h5f['spectrum'][:] #(257, 6, 14641)
YY = np.swapaxes(YYY, 2, 0) #(14641, 6, 257)
Y = np.swapaxes(YY, 2, 1) #(14641, 257, 6)
#YYYY = np.reshape(YY, (14641, 1542)) #(14641, 1542)

### Importing Scikit Learn Parameters

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### Importing Keras Parameters

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Reshape

### Splitting the dataset into training, testing and validation sets

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

### Designing our Model

In [38]:
model = Sequential()
model.add(Dense(8, input_dim = 4, activation = 'relu'))
#model.add(Dense(16, activation = 'relu'))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
#model.add(Dense(128, activation = 'relu'))
#model.add(Dense(256, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
#model.add(Dense(1024, activation = 'relu'))
model.add(Dense(1542, activation = 'relu'))
#model.add(Dense(6, activation = 'relu'))
model.add(Reshape((257, 6)))
#model.output_shape

### Compiling the Model

In [39]:
model.compile(loss = 'mean_squared_logarithmic_error', optimizer = 'adam')

### Fitting the Model

In [40]:
model.fit(X_train, Y_train, epochs = 50, batch_size = 10, validation_split = 0.2)
#model.fit(X, YYY, epochs = 100, batch_size = 10)

Train on 9369 samples, validate on 2343 samples
Epoch 1/50
9369/9369 [==============================] - 12s 1ms/step - loss: 5.8986 - val_loss: 5.3417
Epoch 2/50
9369/9369 [==============================] - 12s 1ms/step - loss: 5.0602 - val_loss: 4.8238
Epoch 3/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.7889 - val_loss: 4.6868
Epoch 4/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.6704 - val_loss: 4.5953
Epoch 5/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.5944 - val_loss: 4.4418
Epoch 6/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.4167 - val_loss: 4.3559
Epoch 7/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.3482 - val_loss: 4.2411
Epoch 8/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.2434 - val_loss: 4.2135
Epoch 9/50
9369/9369 [==============================] - 12s 1ms/step - loss: 4.2359 - val_loss: 4.2040
Epoch 10/50
9369/9369 [==

### Testing our Model

In [41]:
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

### Printing R2 Value

In [42]:
print("The coefficient of determination on the Train set is:\t{:0.3f}".format(r2_score(Y_train.reshape(X_train.shape[0],1542), Y_train_pred.reshape(X_train.shape[0],1542))))
print("The coefficient of determination on the Test set is:\t{:0.3f}".format(r2_score(Y_test.reshape(X_test.shape[0],1542), Y_test_pred.reshape(X_test.shape[0],1542))))

The coefficient of determination on the Train set is:	-79.254
The coefficient of determination on the Test set is:	-77.865
